In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

In [2]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
cols = list(pd.read_csv("/home/rajatsharma/1k.csv", nrows =1))

In [4]:
useless_cols = ['id','timestamp','@version','@timestamp','type','host','message','tags','layer.eth.eth.src', 'layer.eth.eth.dst',
                'layer.json.json.array','layer.json.json.key', 'layer.json.json.value.null','layer.json.json.object',
                'layer.json.json.value.number','layer.json.json.value.false','layer.json.json.member',
                'layer.json.json.value.string','layer.json.json.value.true','src_hostname','langCS','headers','uuid',
               'response_msg','method','status','peer','cookies','download_count','is_virtual','downloads','attack_connection',
               'download_tries','operation_mode','proxy_connection','tftp','app_proto_tc','username','kexAlgs',
               'password','fatt_http','compCS','hassh','hasshAlgorithms','keyAlgs','encCS','version','duration','layer.icmp',
               'macCS','sip','lang','app','fatt_ssh','connection','layer.dns','layer.udp','icmp_type','icmp_code','anomaly',
               'fileinfo','layer.ssh','tls','snmp','sensor','eventid','session','fatt_tls','protocol','layer.nbss','layer.smb',
               'raw_freq','uptime','reason','raw_hits','rdp','link','raw_mtu','os','dist','raw_sig','params','server_ip',
               'client_port','server_port','client_ip','http','subject','mod','ip_rep','ssh','tx_id','layer.data','layer.ssl',
               'host.name','event.original','log.file.path','metadata','log','event','payload','flow','alert','app_proto',
               'dns','layer.tcp','layer.eth','layer.ip','smb','proto','in_iface','flow_id','dest_ip','src_port','dest_port',
               'src_ip','geoip','start_time','end_time','t-pot_ip_ext','t-pot_hostname','t-pot_ip_int','geoip_ext','path',
               'layer.xml.xml.cdata', 'layer.xml.xml.xmlpi.xml.encoding','layer.xml.xml.tag', 'layer.xml.xml.xmlpi.xml.version',
              'layer.xml.xml.attribute','layer.xml.xml.xmlpi.xml','layer.xml.xml.cdatasection',
                'layer.xml.xml.doctype']+[i for i in cols if 'timestamp' in i ]

In [5]:
%%time
temp = pd.read_csv('/home/rajatsharma/1k.csv',low_memory = False,chunksize=10000,
                   usecols=[i for i in cols if i not in useless_cols])
df = pd.concat(temp)
del temp, cols
gc.collect()

CPU times: user 4min 12s, sys: 1min 31s, total: 5min 44s
Wall time: 5min 44s


0

### Changing features type as Boolean

In [6]:
#replacing values with True and False
to_replace = ['layer.tcp.tcp.flags.ack','layer.tcp.tcp.flags.cwr', 'layer.tcp.tcp.flags.ecn',
              'layer.tcp.tcp.flags.fin','layer.tcp.tcp.flags.push','layer.tcp.tcp.flags.reset',
              'layer.tcp.tcp.flags.syn','layer.tcp.tcp.flags.urg','layer.ip.ip.flags.df', 'layer.ip.ip.flags.mf']
cols = df.columns
#Replacing All None with np.nan
def is_unique(s):
    a = s.to_numpy() 
    return (a[0] == a).all()

def to_bool(lst):
    for i in lst:
        if i in cols:
            df[i].replace([1.0], True, inplace= True)
            df[i].replace([0.0], False, inplace= True)
    for i in cols:
        if is_unique(df[i]) == False:
            df[i].replace('None', np.nan,inplace= True)
            df[i].replace('nan', np.nan,inplace = True)
    if 'layer.tcp.tcp.options.sack_perm' in cols:
        df['layer.tcp.tcp.options.sack_perm'].fillna(False,inplace= True)
        df['layer.tcp.tcp.options.sack_perm'].replace('04:02', True, inplace= True)
    return df

In [7]:
%%time
df = to_bool(to_replace)
del to_bool,to_replace,cols
gc.collect()

CPU times: user 1min 8s, sys: 151 ms, total: 1min 8s
Wall time: 1min 8s


0

### To find  which feature how much missing Values

In [8]:
#Check in which columns how many null values
def null_values(data):
    missing_val = data.isnull().sum()
    missing_perc = 100 * data.isnull().sum() / len(data)
    missing_data = pd.concat([missing_val, missing_perc], axis=1)
    missing_data = missing_data.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    missing_data = missing_data[
        missing_data.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(7)
    print ("Dataframe has " + str(data.shape[1]) + " columns.\n" 
           "There are " + str(missing_data.shape[0]) +
           " columns that have missing values.")
    return missing_data

In [9]:
miss_values = null_values(df)

Dataframe has 1647 columns.
There are 1646 columns that have missing values.


#### Filling Missings Values

In [10]:
df.drop(miss_values[miss_values['% of Total Values']==100].index,axis = 'columns',inplace = True)
del miss_values
gc.collect()

0

In [11]:
df.drop(df[df['alert.severity'] == 0].index, axis = 'rows',inplace = True)

In [12]:
df[['alert.signature','alert.category']] = df[['alert.signature','alert.category']].fillna('not_alert')
df.app_linkage.fillna('unknown application layer',inplace = True) 

In [40]:
feat93 = pd.read_excel('/home/rajatsharma/DSA/lcols.xlsx')

In [41]:
feat93

,Cols
0,app_linkage
1,layer.ip._ws.expert.group
2,layer.ip._ws.expert.message
3,layer.ip._ws.expert.severity
4,layer.ip.ip.dsfield.dscp
5,layer.ip.ip.dsfield.ecn
6,layer.ip.ip.flags.df
7,layer.ip.ip.flags.mf
8,layer.ip.ip.frag_offset
9,layer.ip.ip.hdr_len
